In [6]:
import pandas as pd
import numpy as np

The postal code data was copied in an excel sheet. Installing the xlrd package in the cell below as it is required to read excel sheet. 

In [7]:
pip install xlrd

Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [9]:
import xlrd
import requests

In [10]:
df = pd.read_excel('CanadaPostalCodes.xlsx')

Dropping the rows from the dataframe where the Bororugh column is equal to 'Not assigned'

In [11]:
df = df.drop(df[df['Borough'] == 'Not assigned'].index)

Setting the Neighborhood of the rows that are 'Not assigned' to the corresponding value of the 'Borough' column

In [12]:
df.loc[df.Neighborhood == 'Not assigned', 'Neighborhood'] = df.loc[df.Neighborhood == 'Not assigned', 'Borough']

Merging the rows that have the same values for the 'PostalCode' and the 'Borough' columns

In [13]:
df = df.groupby(["PostalCode", "Borough"]).agg( ','.join)

In [14]:
df = df.reset_index()
df.shape

(103, 3)

Copied the data from the csv at the URL at https://cocl.us/Geospatial_data in an Excel sheet and renamed the column 'Postal code' to 'PostalCode' same as in the excel sheet CanadaPostalCodes.xlsx. Loaded the data in a dataframe called geodf

In [15]:
geodf = pd.read_excel('geocoder.xlsx')

Concating the data in the dataframe 'geodf' into the dataframe 'df' based on the column 'PostalCode'

In [16]:
result = pd.concat([df, geodf], axis = 1)
result.head()

,PostalCode,Borough,Neighborhood,PostalCode,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",M1B,43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",M1C,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476


In [17]:
CLIENT_ID = 'C1UYVLF4RL5AWWT22ISOZIFGKGELWO3MKB345KNQVCAPKYX3'
CLIENT_SECRET = 'HPP5M3BSQE3DGNEPI4HHIK5LHNWAV0FIW23ZMDQWBTBM5M0H'
VERSION = '20180605'
LIMIT = 100

Filtering the records of the dataframe for the Toronto borough

In [18]:
toronto_neighborhoods = []
for index, row in result.iterrows():
    if 'Toronto' in row['Borough'] and row['Borough'] not in toronto_neighborhoods:
        toronto_neighborhoods.append(row['Borough']) 
toronto_data = result.loc[df['Borough'].isin(toronto_neighborhoods)]
toronto_data.head()

,PostalCode,Borough,Neighborhood,PostalCode,Latitude,Longitude
37,M4E,East Toronto,The Beaches,M4E,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West,Riverdale",M4K,43.679557,-79.352188
42,M4L,East Toronto,"The Beaches West,India Bazaar",M4L,43.668999,-79.315572
43,M4M,East Toronto,Studio District,M4M,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,M4N,43.728020,-79.388790


In [19]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Getting the top 100 venues for all the neighborhoods in Toronto within a radius of 500 metres

In [20]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

The Beaches
The Danforth West,Riverdale
The Beaches West,India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park,Summerhill East
Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West
Rosedale
Cabbagetown,St. James Town
Church and Wellesley
Harbourfront
Ryerson,Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide,King,Richmond
Harbourfront East,Toronto Islands,Union Station
Design Exchange,Toronto Dominion Centre
Commerce Court,Victoria Hotel
Roselawn
Forest Hill North,Forest Hill West
The Annex,North Midtown,Yorkville
Harbord,University of Toronto
Chinatown,Grange Park,Kensington Market
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place,Underground city
Christie
Dovercourt Village,Dufferin
Little Portugal,Trinity
Brockton,Exhibition Place,Parkdale Village
High Park,The Junction South
Parkdale,Roncesvalles
Runnymede

Checking the size of the resulting dataframe

In [21]:
print(toronto_venues.shape)
toronto_venues.head()

(1726, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West,Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


Checking how many nenues were returned for each neigborhood

In [22]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide,King,Richmond",100,100,100,100,100,100
Berczy Park,55,55,55,55,55,55
"Brockton,Exhibition Place,Parkdale Village",23,23,23,23,23,23
Business Reply Mail Processing Centre 969 Eastern,15,15,15,15,15,15
"CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara",17,17,17,17,17,17
"Cabbagetown,St. James Town",49,49,49,49,49,49
Central Bay Street,87,87,87,87,87,87
"Chinatown,Grange Park,Kensington Market",92,92,92,92,92,92
Christie,16,16,16,16,16,16


Checking the number of unique categories that can be curated from each of the returned values

In [23]:
 print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 240 uniques categories.


Analyzing each Neignborhood

In [24]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Examing each dataframe size

In [25]:
toronto_onehot.shape

(1726, 240)

Grouping rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [26]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,"Adelaide,King,Richmond",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,...,0.000000,0.00000,0.00,0.020000,0.000000,0.000000,0.010000,0.000000,0.000000,0.01
1,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.00,0.018182,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
2,"Brockton,Exhibition Place,Parkdale Village",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
3,Business Reply Mail Processing Centre 969 Eastern,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",0.000000,0.058824,0.058824,0.058824,0.058824,0.176471,0.117647,0.000000,0.000000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
5,"Cabbagetown,St. James Town",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
6,Central Bay Street,0.011494,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.011494,0.000000,...,0.000000,0.00000,0.00,0.011494,0.000000,0.000000,0.011494,0.000000,0.000000,0.00
7,"Chinatown,Grange Park,Kensington Market",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.00,0.054348,0.000000,0.054348,0.010870,0.000000,0.000000,0.00
8,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
9,Church and Wellesley,0.022989,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.011494,0.000000,...,0.000000,0.00000,0.00,0.000000,0.000000,0.011494,0.000000,0.011494,0.011494,0.00


Confirming the new size

In [27]:
toronto_grouped.shape

(39, 240)

Printing each Neighborhood with five most common venues

In [28]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
         venue  freq
0  Coffee Shop  0.08
1         Café  0.04
2   Steakhouse  0.04
3          Bar  0.04
4   Restaurant  0.03


----Berczy Park----
          venue  freq
0   Coffee Shop  0.09
1  Cocktail Bar  0.05
2   Cheese Shop  0.04
3        Bakery  0.04
4    Steakhouse  0.04


----Brockton,Exhibition Place,Parkdale Village----
            venue  freq
0            Café  0.13
1       Nightclub  0.09
2  Breakfast Spot  0.09
3     Coffee Shop  0.09
4   Burrito Place  0.04


----Business Reply Mail Processing Centre 969 Eastern----
           venue  freq
0    Pizza Place  0.07
1        Butcher  0.07
2  Garden Center  0.07
3        Brewery  0.07
4  Auto Workshop  0.07


----CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara----
              venue  freq
0   Airport Service  0.18
1  Airport Terminal  0.12
2   Harbor / Marina  0.06
3           Airport  0.06
4          Boutique  0.06


----Cabbagetown,St. James

Sorting the venues in descending order

In [30]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Creating a new dataframe and displaying the top 10 venues for each neighborhood.

In [31]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Café,Bar,Steakhouse,Restaurant,Burger Joint,Sushi Restaurant,Asian Restaurant,Hotel,Thai Restaurant
1,Berczy Park,Coffee Shop,Cocktail Bar,Steakhouse,Beer Bar,Bakery,Cheese Shop,Farmers Market,Café,Seafood Restaurant,Japanese Restaurant
2,"Brockton,Exhibition Place,Parkdale Village",Café,Coffee Shop,Breakfast Spot,Nightclub,Intersection,Bar,Italian Restaurant,Stadium,Climbing Gym,Furniture / Home Store
3,Business Reply Mail Processing Centre 969 Eastern,Pizza Place,Auto Workshop,Skate Park,Park,Light Rail Station,Farmers Market,Fast Food Restaurant,Burrito Place,Butcher,Restaurant
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Airport Service,Airport Terminal,Harbor / Marina,Bar,Coffee Shop,Rental Car Location,Sculpture Garden,Boutique,Boat or Ferry,Plane


#### Clustering each Neighborhood

Running K means to cluster the neighborhoods into 5 clusters

In [361]:
pip install sklearn

     |████████████████████████████████| 11.0MB 6.4MB/s eta 0:00:01
     |████████████████████████████████| 296kB 11.8MB/s eta 0:00:01
     |████████████████████████████████| 28.4MB 8.4MB/s eta 0:00:01
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1316 sha256=9f5faf091e928bd446443b16ea56cf9d87c1da318e8d650488b4c8acd8b1e17a
  Stored in directory: /Users/pmukher@us.ibm.com/Library/Caches/pip/wheels/76/03/bb/589d421d27431bcd2c6da284d5f2286c8e3b2ea3cf1594c074
Successfully built sklearn
Note: you may need to restart the kernel to use updated packages.


In [32]:
from sklearn.cluster import KMeans

In [39]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,"Adelaide,King,Richmond",Coffee Shop,Café,Bar,Steakhouse,Restaurant,Burger Joint,Sushi Restaurant,Asian Restaurant,Hotel,Thai Restaurant
1,0,Berczy Park,Coffee Shop,Cocktail Bar,Steakhouse,Beer Bar,Bakery,Cheese Shop,Farmers Market,Café,Seafood Restaurant,Japanese Restaurant
2,0,"Brockton,Exhibition Place,Parkdale Village",Café,Coffee Shop,Breakfast Spot,Nightclub,Intersection,Bar,Italian Restaurant,Stadium,Climbing Gym,Furniture / Home Store
3,0,Business Reply Mail Processing Centre 969 Eastern,Pizza Place,Auto Workshop,Skate Park,Park,Light Rail Station,Farmers Market,Fast Food Restaurant,Burrito Place,Butcher,Restaurant
4,0,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Airport Service,Airport Terminal,Harbor / Marina,Bar,Coffee Shop,Rental Car Location,Sculpture Garden,Boutique,Boat or Ferry,Plane


 Creating a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [41]:
# add clustering labels
toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,PostalCode,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,M4E,East Toronto,The Beaches,M4E,43.676357,-79.293031,0,Trail,Pub,Health Food Store,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Women's Store
41,M4K,East Toronto,"The Danforth West,Riverdale",M4K,43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Furniture / Home Store,Restaurant,Pub,Diner,Caribbean Restaurant,Bakery
42,M4L,East Toronto,"The Beaches West,India Bazaar",M4L,43.668999,-79.315572,0,Park,Sandwich Place,Brewery,Movie Theater,Sushi Restaurant,Ice Cream Shop,Steakhouse,Italian Restaurant,Food & Drink Shop,Pub
43,M4M,East Toronto,Studio District,M4M,43.659526,-79.340923,0,Café,Coffee Shop,Brewery,Gastropub,Bakery,Italian Restaurant,American Restaurant,Yoga Studio,Comfort Food Restaurant,Seafood Restaurant
44,M4N,Central Toronto,Lawrence Park,M4N,43.728020,-79.388790,4,Park,Swim School,Bus Line,Women's Store,Dim Sum Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant


Visualizing the clusters

In [42]:
pip install folium

     |████████████████████████████████| 92kB 1.3MB/s eta 0:00:011
Note: you may need to restart the kernel to use updated packages.


In [43]:
import folium

Getting the latitude and langitude values for Toronto

In [48]:
latitude = max(toronto_merged['Latitude'])
longitude = max(toronto_merged['Longitude'])
print(latitude)
print(longitude)

43.7280205
-79.2930312


In [51]:
pip install matplotlib

     |████████████████████████████████| 13.2MB 6.6MB/s eta 0:00:01
     |████████████████████████████████| 71kB 12.6MB/s eta 0:00:01
     |████████████████████████████████| 122kB 12.3MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [53]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

In [55]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters